<a href="https://colab.research.google.com/github/Daramluv/Google_colab/blob/main/Product_Mix_computer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 26.0 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
#####################################################
#                    Model Formulation
#####################################################

m = gp.Model('computer mix')

Restricted license - for non-production use only - expires 2026-11-23


In [ ]:
#Input Parameters
computer_type = [*range(0,2)]
Computer_type_label =['Basic','XP']

LaborAssembly = [5,6]
LaborTesting = [1,2]
CostComponents =[150,225]
SellingPrice = [300,450]
Demand = [600,1200]
TotalAssemblyhours = 10000
TotalTestinghours = 3000
LACost = 11
LTCost = 15

In [ ]:
#Decision Variable
x = m.addVars(computer_type, vtype=GRB.CONTINUOUS, name=Computer_type_label)

In [ ]:
# Set the Maximize Obijective: Total profit
m.setObjective(gp.quicksum((SellingPrice[i]-CostComponents[i]-LaborAssembly[i]*LACost -LaborTesting[i]*LTCost)*x[i]
                          for i in computer_type),  GRB.MAXIMIZE)

In [ ]:
#Set Non-negative Decision Variable
c1 = m.addConstrs((x[i] >= 0 for i in computer_type), name = 'non-negative production')

# Total Computers produced must be less than or equal to Demand
c2 = m.addConstrs((x[i] <= Demand[i] for i in computer_type), name = 'cannot exceed maximum demand')

# Assemble hours cannot exceed available
c3 = m.addConstr((gp.quicksum(LaborAssembly[i]*x[i] for i in computer_type) <= TotalAssemblyhours), name = 'cannot exceed available assembly hours')

# Testing hours cannot exceed available
c4 = m.addConstr((gp.quicksum(LaborTesting[i]*x[i] for i in computer_type) <= TotalTestinghours), name ='cannot exceed available testing hours')

In [ ]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 2 columns and 8 nonzeros
Model fingerprint: 0x3d53bcff
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [8e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 1e+04]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1500000e+05   2.333333e+02   0.000000e+00      0s
       1    1.9960000e+05   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.996000000e+05


In [ ]:
# Get the Optimal Solution for X
print("The Optimal number of each computer type to be produced for maximum profit\n")
m.printAttr('X')
def comma_value(number):
    return ("{:,}".format(number))
# print("\nTotal profit: $%8.2f" % (m.ObjVal))
print("\nTotal profit: $",comma_value(m.ObjVal))
# m.printAttr('RHS')
# m.printAttr('Slack')

The Optimal number of each computer type to be produced for maximum profit


    Variable            X 
-------------------------
       Basic          560 
          XP         1200 

Total profit: $ 199,600.0


In [ ]:
#Finding amount of Profit generated by computer type
e = m.getObjective()
margin = [e.getCoeff(i) for i in computer_type ]
production = [v.x for v in m.getVars()]
revenue = [margin[i]*production[i] for i in computer_type]
for i in computer_type:
  print("Profit generated by",Computer_type_label[i],"computer is $",comma_value(revenue[i]))

Profit generated by Basic computer is $ 44,800.0
Profit generated by XP computer is $ 154,800.0


In [ ]:
#Finding the number of assembly hours used by computer type
assembly_hour_used = [LaborAssembly[i]*production[i] for i in computer_type]
for i in computer_type:
  print("Number of Assembly hours used by",Computer_type_label[i],"computer is",comma_value(assembly_hour_used[i]))

print("Total Assembly hours used is",comma_value(sum(assembly_hour_used)))

Number of Assembly hours used by Basic computer is 2,800.0
Number of Assembly hours used by XP computer is 7,200.0
Total Assembly hours used is 10,000.0


In [ ]:
#Finding the number of testing hours used by computer type
testing_hour_used = [LaborTesting[i]*production[i] for i in computer_type]
for i in computer_type:
  print("Number of Testing hours used by",Computer_type_label[i],"computer is",comma_value(testing_hour_used[i]))

print("Total Testing hours",comma_value(sum(testing_hour_used)))

Number of Testing hours used by Basic computer is 560.0
Number of Testing hours used by XP computer is 2,400.0
Total Testing hours 2,960.0
